<a href="https://colab.research.google.com/github/ProfAI/tf00/blob/master/4%20-%20Tensorflow%20Avanzato/model_export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Esportare una Rete Neurale
Nel notebook precedente abbiamo visto come fare un backup della Rete Neurale utilizzando i Checkpoint, prima di vedere come re-importare la Rete salvata su disco, vediamo come poter salvare la Rete completa una volta terminato l'addestramento.

## Importiamo i Moduli

In [ ]:
import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from time import time

## Prepariamo i Dati

In [ ]:
dataset = tfds.load('mnist', split='train', shuffle_files=True)

def load_data(dataset, num_samples=None):
  images = []
  labels = []

  for i, example in enumerate(tfds.as_numpy(dataset)):

    if(num_samples!=None and i>=num_samples):
      break

    images.append(example["image"])
    labels.append(example["label"])

  images = np.array(images)
  labels = np.array(labels)

  return images, labels

images, labels = load_data(dataset)
num_classes = np.unique(labels).shape[0]

X = images
y = tf.one_hot(labels, num_classes).numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.0. Subsequent calls will reuse this data.


## Costruiamo la Rete Neurale

In [ ]:
def build_model():
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Lambda(lambda x: tf.reshape(x, [x.shape[0], x.shape[1]*x.shape[2]])))
  model.add(tf.keras.layers.Lambda(lambda x: x/255))
  model.add(tf.keras.layers.Dense(32, activation='relu'))
  model.add(tf.keras.layers.Dense(32, activation='relu'))
  model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

  return model

In [ ]:
model = build_model()
model.fit(X_train, y_train, epochs=20)

Epoch 1/20
1500/1500 [==============================] - 2s 1ms/step - loss: 0.8885 - accuracy: 0.7535
Epoch 2/20
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3583 - accuracy: 0.8971
Epoch 3/20
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3002 - accuracy: 0.9140
Epoch 4/20
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2663 - accuracy: 0.9239
Epoch 5/20
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2417 - accuracy: 0.9306
Epoch 6/20
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2211 - accuracy: 0.9370
Epoch 7/20
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2051 - accuracy: 0.9415
Epoch 8/20
1500/1500 [==============================] - 2s 1ms/step - loss: 0.1912 - accuracy: 0.9459
Epoch 9/20
1500/1500 [==============================] - 2s 1ms/step - loss: 0.1800 - accuracy: 0.9485
Epoch 10/20
1500/1500 [==============================] - 2s 1ms/step - loss: 0.169

In [ ]:
model.evaluate(X_test, y_test)

375/375 [==============================] - 0s 1ms/step - loss: 0.1550 - accuracy: 0.9562


[0.15500508248806, 0.956166684627533]

## Esportiamo la Rete
Esportare la Rete su disco è estremamente semplice, tutto ciò che dobbiamo fare è utilizzare il metodo *save*, specificando il percorso al file h5 che verrà creato per immagazzinare il modello.

In [ ]:
model.save("model_mnist.h5")